In [1]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [2]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [3]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/housing.csv".format(container, storage_account_name))

data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
 -122.27| 37.84| 52.0| 1503.0| 298.0| 690.0| 275.0| 2.6033| 162900.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [5]:
vectors = VectorAssembler(inputCols= ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'], outputCol= 'features')

vectors.setParams(handleInvalid="skip")

vector_data = vectors.transform(data)

In [6]:
features = vector_data.select(["features", "median_house_value"])

features.show()

+--------------------+------------------+
 features|median_house_value|
+--------------------+------------------+
[-122.23,37.88,41...| 452600.0|
[-122.22,37.86,21...| 358500.0|
[-122.24,37.85,52...| 352100.0|
[-122.25,37.85,52...| 341300.0|
[-122.25,37.85,52...| 342200.0|
[-122.25,37.85,52...| 269700.0|
[-122.25,37.84,52...| 299200.0|
[-122.25,37.84,52...| 241400.0|
[-122.26,37.84,42...| 226700.0|
[-122.25,37.84,52...| 261100.0|
[-122.26,37.85,52...| 281500.0|
[-122.26,37.85,52...| 241800.0|
[-122.26,37.85,52...| 213500.0|
[-122.26,37.84,52...| 191300.0|
[-122.26,37.85,52...| 159200.0|
[-122.26,37.85,50...| 140000.0|
[-122.27,37.85,52...| 152500.0|
[-122.27,37.85,52...| 155500.0|
[-122.26,37.84,50...| 158700.0|
[-122.27,37.84,52...| 162900.0|
+--------------------+------------------+
only showing top 20 rows

In [7]:
(trainingData, testData) = features.randomSplit([0.7, 0.3])

In [8]:
lr = LinearRegression(labelCol="median_house_value", featuresCol="features")

In [9]:
model = lr.fit(trainingData)

In [10]:
print("Coefficients:", model.coefficients)
print("Intercept:", model.intercept)

Coefficients: [-43428.0460926,-43501.9876568,1161.48364961,-7.38631867014,116.632812883,-43.2570480095,51.3832406282,39553.9821166]
Intercept: -3629081.6014177697

In [11]:
model_summary = model.summary

print("RMSE", model_summary.rootMeanSquaredError)
print("R^2:", model_summary.r2)

RMSE 70284.02153900571
R^2: 0.6310748629141505

In [12]:
model.save("dbfs:/FileStore/lr_model.model")

In [13]:
dbutils.fs.ls("dbfs:/FileStore")

Out[ 20 ]: 
[FileInfo(path='dbfs:/FileStore/import-stage/', name='import-stage/', size=0),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0),
 FileInfo(path='dbfs:/FileStore/lr_model.model/', name='lr_model.model/', size=0),
 FileInfo(path='dbfs:/FileStore/plots/', name='plots/', size=0),
 FileInfo(path='dbfs:/FileStore/profiles/', name='profiles/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0)]

In [14]:
loaded_model = LinearRegressionModel.load("dbfs:/FileStore/lr_model.model")

In [15]:
print("Coefficients:", loaded_model.coefficients)
print("Intercept:", loaded_model.intercept)

Coefficients: [-43428.0460926,-43501.9876568,1161.48364961,-7.38631867014,116.632812883,-43.2570480095,51.3832406282,39553.9821166]
Intercept: -3629081.6014177697